In [8]:
!pip install pyTelegramBotAPI

import telebot
from telebot import types
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk import word_tokenize, pos_tag

from string import punctuation
import re


import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

import spacy

!pip install stop-words
from stop_words import get_stop_words

from textblob import TextBlob

from bs4 import BeautifulSoup
import requests
import json
from urllib.parse import urlparse


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [35]:
bot = telebot.TeleBot('5260450850:AAGkp80bHug3CzO0AVj41IEoVNDocOJLfGk');

In [10]:
def parser(link):
    #link='https://www.foxnews.com/politics/walt-disney-would-be-rolling-over-in-his-grave-over-companys-wokeness-florida-park-goer-says'
    req = requests.get(link).text
    soup = BeautifulSoup(req, "lxml")
    head = soup.find('head')
    zagolovok = head.find('title').text
    textp = soup.find_all('div', attrs={'class': "article__text"})
    if (len(textp)) == 0:
        textp = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        text = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        textp = soup.find_all('p')
    
    # domain = urlparse(link).netloc
    # if (int(domain.find("www.")) != -1):
    #   domain = domain[4:]
    # url = link.split("//")[-1].split("/")[0].split('?')[0]
    # show = "https://input.payapi.io/v1/api/fraud/domain/age/" + domain
    # r = requests.get(show, verify = False)
    # soupp = BeautifulSoup(r.text, "lxml")
    # if requests.get(link).status_code ==200:
    #     age = soupp.find('body').text
    #     str = int(age.find("Reg"))
    #     end = int(age.find("}"))
    #     # print(age[str:(end - 1)])
    #     age=age[str:(end - 1)]
    #     data = r.text
    #     jsonToPython = json.loads(data)
    # else:
    #   age=np.nan

    for j in range(len(textp)):
      textp[j]=textp[j].text

    textp=''.join(textp)
    zagolovok=''.join(zagolovok)

    return textp, zagolovok
    

In [11]:
from google.colab import files
uploaded = files.upload()

Saving итоговая модель.h5 to итоговая модель.h5
Saving модель_для_анализа_заголовка.h5 to модель_для_анализа_заголовка.h5


In [12]:
# анализ заголовка
def headlineAnalis(headline):
    model_for_headline = keras.models.load_model('модель_для_анализа_заголовка.h5')

    df_h = pd.DataFrame({'Headline' : [headline]})

    oh_headlines = [one_hot(words, 5000) for words in df_h['Headline']]
    x = pad_sequences(oh_headlines, padding='pre', maxlen=30)

    headLines = model_for_headline.predict(x)
    df_h.Headline = headLines

    return df_h

In [13]:
# удаление пунктуации и привеение к нижнему регистру

def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ',
             43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ',
             63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def cleanText(text):
    text = text.lower()
    
    text = remove_punct(text)

    signs = ['?', ',', ':', '!', '.', ';', '«', '»', '(', ')', '*', '+', '-', '^', '|', '=', "’", "‘",  '”', '—', '@', '>', '<', '–', '&', '%', '•', '/', '“']
    for punct_sign in signs:
        text = text.replace(punct_sign, ' ')

    text = text.replace('\n', '')
    text = text.replace('\r', '')
    text = re.sub(r"\d+", "", text, flags=re.UNICODE)
    text = text.replace("  " , " ")
    text = text.replace("  " , " ")

    return text

In [14]:
# подсчет количества наречий

def amountAdverbs(text):
    amountADV = 0
    tokens = word_tokenize(text)
    tags = pos_tag(tokens)
    for i in tags:
        if len(i) == 2:
            if i[1] == 'RB':
                amountADV += 1
        else:
            continue
    return amountADV

In [15]:
# лемматизация и удаление стоп-слов

def removeStopWords_and_lemmitizeText(text):
    lemmatizer = spacy.load('en', disable=['parser', 'ner'])
    doc = lemmatizer(text)
    new_text = " ".join([token.lemma_ for token in doc\
                         if token.lemma_ not in get_stop_words('english')\
                         and (token.lemma_ != "-PRON-")\
                         and (len(token.lemma_) != 1 or token.lemma_ == 'i')])
    # new_text = new_text.replace("-PRON-", '')
    # new_text = new_text.replace("s", '')
    return new_text

In [16]:
# тональность и субъективность

def polarityAndSubjectivityOfText(text):
    pol = TextBlob(text).polarity
    sub = TextBlob(text).subjectivity
    return pol, sub

In [17]:
# векторизация

def toVecText(text):

    d = {'text': [text]}
    df1 = pd.DataFrame(d)

    oh_text = [one_hot(words, 5000) for words in df1.text]
    embedded_doc = pad_sequences(oh_text, padding='pre', maxlen=800)
    df_text = pd.DataFrame(embedded_doc)

    return df_text

# oh_text = one_hot(text, 5000)
# embedded_doc = pad_sequences(oh_text, padding='pre', maxlen=800)
# df_text = pd.DataFrame(embedded_doc)
# df_new = pd.concat([df.drop(['Body_02'], axis = 1), df_text], axis = 1)

In [18]:
# from numpy.ma.core import logical_not
def allTextPreparation(text):

    # чистим от пунктуации + нижгий регистр
    text = cleanText(text)

    # считаем количество наречий
    amountAdv = amountAdverbs(text)

    # лемматизация и удаление стоп-слов
    text = removeStopWords_and_lemmitizeText(text)

    # полярность и субъективность
    l_pol_sub = polarityAndSubjectivityOfText(text)

    polarity = l_pol_sub[0]
    subjectivity = l_pol_sub[1]

    # векторизация текста
    vec_text = toVecText(text)

    data_ = {'Amount Adverbs' : [amountAdv], 
             'Polarity' : [polarity],
             'Subjectivity' : [subjectivity]}

    df_features = pd.DataFrame(data_)

    df_new = pd.concat([df_features, vec_text], axis = 1)

    return df_new


In [19]:
def dataPreparation(data):
    # data = [text, headline]
    text = data[0]
    headline = data[1]
    df_text = allTextPreparation(text)
    df_headline = headlineAnalis(headline)

    df = pd.concat([df_headline, df_text], axis = 1)

    return df


In [20]:
def dataPrediction(data_df):
    finalModel = keras.models.load_model('итоговая модель.h5')
    result = finalModel.predict(data_df)
    result = int(np.around(result))

    if result == 0:
        return "С высокой долей вероятности новость - фейк."
    elif result == 1:
        return "С высокой долей вероятности новость не является фейком."
    

In [21]:
def fakeNewsChecking(url):
    # отправляем url в парсер, получаем данные
    data = parser(url)
    df = dataPreparation(data)
    res = dataPrediction(df)
    return res

In [36]:

@bot.message_handler(commands=["start", "restart"])
def start(m, res=False):
        bot.send_message(m.chat.id, 'Привет, пользователь!\n\nМеня зовут FakeBot. Я защищу тебя от фейков в сети. Для этого мне лишь потребуется ссылка на подозрительную новость, нуждающуюся в проверке.\n\nСписок команд:\
        \n/restart – перезапустить ботa\
        \n/help – показать гид по командам\
        \n/check – проверить новость\
        \n/more – узнать больше обо мне и моих создателях')
@bot.message_handler(commands=["help"])
def help(message):
  bot.send_message(message.from_user.id, 'Список команд:\
  \n/restart – перезапустить ботa\
  \n/help – показать гид по командам\
  \n/check – проверить новость\
  \n/more – узнать больше обо мне и моих создателях')
# @bot.message_handler(commands=["start"])
# def start(m, res=False):
#         markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
#         item1=types.KeyboardButton("Начать")
#         item2=types.KeyboardButton("Информация")
#         item3=types.KeyboardButton("Помощь")
#         markup.add(item1)
#         markup.add(item2)
#         markup.add(item3)
#         bot.send_message(m.chat.id, 'Нажми: \nНачать(чтобы воспользоваться программой)\nИнформация(чтобы узанть о нас)\nПомощь(чтобы узнать о боте)',  reply_markup=markup)

In [37]:
@bot.message_handler(commands=["more"])
def help(message):
  bot.send_message(message.from_user.id, 'Меня разработала пара ребят, разработчиков-любителей, из лицея 1511 при МИФИ в городе Москве. Они очень старались и надеются, что этот бот будет тебе полезен. 😊')
@bot.message_handler(commands="check")
def rabota(message):
  bot.send_message(message.from_user.id, 'Введи ссылку на новость')
  bot.register_next_step_handler(message, test);
def test(message):
  link=message.text
  if requests.get(link).status_code==200:
    link=message.text
    bot.send_message(message.chat.id, 'Подожди несколько секунд, идут сложные вычисления...')
    predict=fakeNewsChecking(link)
    bot.send_message(message.chat.id,text=predict)
  else:
    bot.send_message(message.chat.id,'C этой ссылкой что-то не так. Возможно, сайт не отвечает, или страница больше не существует.')

# @bot.message_handler(func=lambda message: message.text=='Начать')
# def rabota(message):
#   bot.send_message(message.from_user.id, 'Введите ссылку на новость')
#   bot.register_next_step_handler(message, test);
# def test(message):
#     bot.send_message(message.chat.id, 'Подожди несколько секунд, идут сложные вычисления...')
#     link=message.text
#     predict=fakeNewsChecking(link)
#     bot.send_message(message.chat.id,text=predict)
# def proga(m):
#   bot.send_message(m.from_user.id, 'ссылка обрабатывается')
# @bot.message_handler(func=lambda message: message.text=='Информация')
# def info(message):
#   bot.send_message(message.from_user.id, "Это проект учащихся IT-класса лицея №1511 при МИФИ, созданный для защиты от фейковых новостей в сети.")
# @bot.message_handler(func=lambda message: message.text=='Помощь')
# def help(message):
#   bot.send_message(message.from_user.id, "Если у вас есть вопросы по боту или работе программы, то задать их можно здесь @miCherex")

bot.polling() 